In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cross_validation import KFold
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from pandas import Series, DataFrame
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn import tree

from xgboost.sklearn import XGBRegressor

/home/vaibhavmathur/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge


In [4]:
train=pd.read_csv(r"/home/vaibhavmathur/Downloads/Train_Big_Mart.csv")

In [5]:
test=pd.read_csv(r"/home/vaibhavmathur/Downloads/Test_Big_Mart.csv")
a=test.Item_Identifier
b=test.Outlet_Identifier

In [6]:
train.Item_Fat_Content.unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [7]:
train.Item_Type.unique()

array(['Dairy', 'Soft Drinks', 'Meat', 'Fruits and Vegetables',
       'Household', 'Baking Goods', 'Snack Foods', 'Frozen Foods',
       'Breakfast', 'Health and Hygiene', 'Hard Drinks', 'Canned',
       'Breads', 'Starchy Foods', 'Others', 'Seafood'], dtype=object)

In [8]:
Y=train.Item_Outlet_Sales


In [9]:
train['Item_Visibility']=train['Item_Visibility'].replace(0,np.mean(train['Item_Visibility']))
train['Outlet_Establishment_Year']=2013-train['Outlet_Establishment_Year']
train['Item_Weight'].fillna((train['Item_Weight'].mean()), inplace=True)
train['Outlet_Size'].fillna('Small',inplace=True)
train['Item_Fat_Content']=train['Item_Fat_Content'].replace(('low fat','LF'),'Low Fat')
train['Item_Fat_Content']=train['Item_Fat_Content'].replace('reg','Regular')
train['Item_Type']=train['Item_Type'].replace(('Dairy','Soft Drinks','Meat','Fruits and Vegetables','Snack Foods',
                                              'Breakfast','Breads','Seafood'),'Perishable')
train['Item_Type']=train['Item_Type'].replace(('Baking Goods','Hard Drinks','Canned','Starchy Foods','Frozen Foods'),'NonPerishable')
train['Item_Type']=train['Item_Type'].replace(('Health and Hygiene','Household','Others'),'Others')

In [10]:
train['PPUW']=train['Item_MRP']/train['Item_Weight']

In [11]:
sqrt_transformed=train['Item_Visibility'].apply(np.sqrt)
train['Item_Visibility']=sqrt_transformed

sqrt_transformedPP=train['PPUW'].apply(np.sqrt)
train['PPUW']=sqrt_transformedPP

In [12]:
mylist = list(train.select_dtypes(include=['object']).columns)
mylist.remove("Item_Identifier")
mylist

['Item_Fat_Content',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

In [13]:
dummies = pd.get_dummies(train[mylist], prefix= mylist)
train.drop(mylist, axis=1, inplace=True)
train=pd.concat([train,dummies], axis=1)

In [14]:
train.shape
train.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year', 'Item_Outlet_Sales', 'PPUW',
       'Item_Fat_Content_Low Fat', 'Item_Fat_Content_Regular',
       'Item_Type_NonPerishable', 'Item_Type_Others', 'Item_Type_Perishable',
       'Outlet_Identifier_OUT010', 'Outlet_Identifier_OUT013',
       'Outlet_Identifier_OUT017', 'Outlet_Identifier_OUT018',
       'Outlet_Identifier_OUT019', 'Outlet_Identifier_OUT027',
       'Outlet_Identifier_OUT035', 'Outlet_Identifier_OUT045',
       'Outlet_Identifier_OUT046', 'Outlet_Identifier_OUT049',
       'Outlet_Size_High', 'Outlet_Size_Medium', 'Outlet_Size_Small',
       'Outlet_Location_Type_Tier 1', 'Outlet_Location_Type_Tier 2',
       'Outlet_Location_Type_Tier 3', 'Outlet_Type_Grocery Store',
       'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2',
       'Outlet_Type_Supermarket Type3'],
      dtype='object')

In [15]:
X = train.drop(["Item_Identifier","Item_Outlet_Sales"], axis=1)
X.shape

(8523, 30)

In [16]:
#lreg = LinearRegression()
#rreg=Ridge(alpha=0.05, normalize=True)
#lasreg=Lasso(alpha=0.05, normalize=True)


In [17]:
X.columns

Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year', 'PPUW', 'Item_Fat_Content_Low Fat',
       'Item_Fat_Content_Regular', 'Item_Type_NonPerishable',
       'Item_Type_Others', 'Item_Type_Perishable', 'Outlet_Identifier_OUT010',
       'Outlet_Identifier_OUT013', 'Outlet_Identifier_OUT017',
       'Outlet_Identifier_OUT018', 'Outlet_Identifier_OUT019',
       'Outlet_Identifier_OUT027', 'Outlet_Identifier_OUT035',
       'Outlet_Identifier_OUT045', 'Outlet_Identifier_OUT046',
       'Outlet_Identifier_OUT049', 'Outlet_Size_High', 'Outlet_Size_Medium',
       'Outlet_Size_Small', 'Outlet_Location_Type_Tier 1',
       'Outlet_Location_Type_Tier 2', 'Outlet_Location_Type_Tier 3',
       'Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type1',
       'Outlet_Type_Supermarket Type2', 'Outlet_Type_Supermarket Type3'],
      dtype='object')

In [18]:
#from sklearn import preprocessing
#scaler = preprocessing.StandardScaler().fit(X)
#scaler
#StandardScaler(copy=True, with_mean=True, with_std=True)
#scaler.mean_ 

In [19]:
#scaler.scale_         
#scaler.transform(X)                           

In [20]:
#rfe = RFE(lasreg, 15)
#rfe = rfe.fit(X, Y)
#print(rfe.support_)
#print(rfe.ranking_)

In [21]:
#cols=['Item_Visibility', 'Item_MRP','PPUW', 'Item_Fat_Content_Low Fat','Item_Type_Others','Outlet_Identifier_OUT010',
#'Outlet_Identifier_OUT017','Outlet_Identifier_OUT018','Outlet_Identifier_OUT027', 'Outlet_Identifier_OUT035',
 #      'Outlet_Identifier_OUT045', 'Outlet_Identifier_OUT046',
  #     'Outlet_Identifier_OUT049',  'Outlet_Type_Supermarket Type2','Outlet_Type_Grocery Store']

In [22]:
x_train, x_cv, y_train, y_cv = train_test_split(X,train.Item_Outlet_Sales, test_size =0.3)

In [23]:
#Linear Regression
#lreg = LinearRegression()
#lreg.fit(x_train,y_train)
#pred_cv=lreg.predict(x_cv)

In [24]:
#Ridge Regression
#rreg=Ridge(alpha=0.05)
#rreg=rreg.fit(x_train, y_train)
#pred_cv=rreg.predict(x_cv)

In [25]:
#Lasso Regression

#lasreg=Lasso(alpha=0.15, normalize=True)
#lasreg=lasreg.fit(x_train,y_train)
#pred_cv=lasreg.predict(x_cv)

In [26]:
#mse = np.mean((pred_cv - y_cv)**2)

In [27]:
#mse

In [28]:
#lasreg.score(x_cv,y_cv)

In [29]:
#Feature Engineering on Test Data

In [30]:
test['Item_Visibility']=test['Item_Visibility'].replace(0,np.mean(test['Item_Visibility']))
test['Outlet_Establishment_Year']=2013-test['Outlet_Establishment_Year']
test['Item_Weight'].fillna((test['Item_Weight'].mean()), inplace=True)
test['Outlet_Size'].fillna('Small',inplace=True)
test['Item_Fat_Content']=test['Item_Fat_Content'].replace(('low fat','LF'),'Low Fat')
test['Item_Fat_Content']=test['Item_Fat_Content'].replace('reg','Regular')
test['Item_Type']=test['Item_Type'].replace(('Dairy','Soft Drinks','Meat','Fruits and Vegetables','Snack Foods',
                                              'Breakfast','Breads','Seafood'),'Perishable')
test['Item_Type']=test['Item_Type'].replace(('Baking Goods','Hard Drinks','Canned','Starchy Foods','Frozen Foods'),'NonPerishable')
test['Item_Type']=test['Item_Type'].replace(('Health and Hygiene','Household','Others'),'Others')
test['PPUW']=test['Item_MRP']/test['Item_Weight']

In [31]:
sqrt_transformed=train['Item_Visibility'].apply(np.sqrt)
train['Item_Visibility']=sqrt_transformed

sqrt_transformedPP=train['PPUW'].apply(np.sqrt)
train['PPUW']=sqrt_transformedPP

In [32]:
mylist1 = list(test.select_dtypes(include=['object']).columns)
mylist1.remove("Item_Identifier")



In [33]:
dummies = pd.get_dummies(test[mylist1], prefix= mylist1)
test.drop(mylist1, axis=1, inplace=True)
test=pd.concat([test,dummies], axis=1)

In [34]:
y=train.Item_Outlet_Sales
x=train.drop(["Item_Outlet_Sales","Item_Identifier"], axis=1)
test=test.drop("Item_Identifier", axis=1)

In [35]:
#scaler = preprocessing.StandardScaler().fit(test)
#scaler
#StandardScaler(copy=True, with_mean=True, with_std=True)
#scaler.mean_ 
#scaler.scale_         
#$scaler.transform(test) 

In [36]:
#Linear on test
#lreg = LinearRegression()
#lreg.fit(x,y)
#pred_cv=lreg.predict(test)

In [37]:
#Ridge on test
#rreg=Ridge(alpha=0.05)
#rreg=rreg.fit(x, y)
#pred_cv=rreg.predict(test)

In [38]:
#Lasso on test

In [39]:
#lasreg=Lasso(alpha=0.15, normalize=True)
#lasreg=lasreg.fit(x,y)


In [40]:
#outdf=pd.DataFrame({"Item_Identifier":a,"Outlet_Identifier":b,"Item_Outlet_Sales":pred_cv})

In [41]:
#outdf.to_csv("SubLasso.csv")

In [42]:
#test.describe()

In [43]:
sqrt_transformed=train['Item_Visibility'].apply(np.sqrt)
train['Item_Visibility']=sqrt_transformed

In [44]:
#model = tree.DecisionTreeClassifier(criterion='gini')

In [45]:
#model = tree.DecisionTreeRegressor(max_depth=5)

In [46]:
#model.fit(x, y)


In [47]:
#model.score(x, y)

In [48]:
#predicted= model.predict(test)

In [49]:
#from sklearn.ensemble import RandomForestRegressor

In [50]:
#model= RandomForestRegressor(n_estimators=1000, max_depth=5)

In [51]:
#model.fit(x, y)

#modelxgb = XGBRegressor( learning_rate = 0.1 , n_estimators = 1000 , max_depth = 5, min_child_weight = 1, gamma = 0, subsample = 0.8,colsample_bytree=0.8,objective= 'reg:linear',nthread=4,scale_pos_weight=1,seed=27)
#modelxgb.fit(x,y)


In [52]:
#predicted=modelxgb.predict(test)

In [59]:
from tpot import TPOTRegressor
X_train, X_test, y_train, y_test = train_test_split(x, y,
 train_size=0.75, test_size=0.25)

tpot = TPOTRegressor(generations=30, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_boston_pipeline.py')

Optimization Progress:   6%|▋         | 100/1550 [04:27<1:21:13,  3.36s/pipeline]

Generation 1 - Current best internal CV score: -1174081.7379502184


Optimization Progress:  10%|▉         | 150/1550 [10:25<1:21:55,  3.51s/pipeline] 

Generation 2 - Current best internal CV score: -1159110.960836057


Optimization Progress:  13%|█▎        | 200/1550 [14:32<2:11:31,  5.85s/pipeline]

...

Generation 28 - Current best internal CV score: -1154603.5898996487


Optimization Progress:  97%|█████████▋| 1500/1550 [1:21:36<01:21,  1.63s/pipeline]

Generation 29 - Current best internal CV score: -1154603.5898996487


Generation 30 - Current best internal CV score: -1154603.5898996487

Best pipeline: ExtraTreesRegressor(SelectPercentile(input_matrix, percentile=45), bootstrap=True, max_features=0.7500000000000001, min_samples_leaf=19, min_samples_split=4, n_estimators=100)
-1188490.1388257665


True

In [60]:
tpot_pred = tpot.predict(test)
outdf=pd.DataFrame({"Item_Identifier":a,"Outlet_Identifier":b,"Item_Outlet_Sales":tpot_pred})
outdf.to_csv("Tpot.csv")